In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [21]:
import pandas as pd

from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import spacy
from spacy.tokenizer import Tokenizer

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [22]:
X_train = pd.read_pickle("X_train.pkl")
X_test = pd.read_pickle("X_test.pkl")
y_train = pd.read_pickle("y_train.pkl")
y_test = pd.read_pickle("y_test.pkl")

In [23]:
X_train.head()

,joke_x,ambiguous_words,best_score_similarity_words,antonyms,longest_word,speical_chars,MEAN_CHARACTERS_PER_WORD(joke),UNIVERSAL_SENTENCE_ENCODER(joke)[0],UNIVERSAL_SENTENCE_ENCODER(joke)[1],UNIVERSAL_SENTENCE_ENCODER(joke)[2],UNIVERSAL_SENTENCE_ENCODER(joke)[3],UNIVERSAL_SENTENCE_ENCODER(joke)[4],UNIVERSAL_SENTENCE_ENCODER(joke)[5],UNIVERSAL_SENTENCE_ENCODER(joke)[6],UNIVERSAL_SENTENCE_ENCODER(joke)[7],UNIVERSAL_SENTENCE_ENCODER(joke)[8],UNIVERSAL_SENTENCE_ENCODER(joke)[9],UNIVERSAL_SENTENCE_ENCODER(joke)[10],UNIVERSAL_SENTENCE_ENCODER(joke)[11],UNIVERSAL_SENTENCE_ENCODER(joke)[12],UNIVERSAL_SENTENCE_ENCODER(joke)[13],UNIVERSAL_SENTENCE_ENCODER(joke)[14],UNIVERSAL_SENTENCE_ENCODER(joke)[15],UNIVERSAL_SENTENCE_ENCODER(joke)[16],UNIVERSAL_SENTENCE_ENCODER(joke)[17],UNIVERSAL_SENTENCE_ENCODER(joke)[18],UNIVERSAL_SENTENCE_ENCODER(joke)[19],UNIVERSAL_SENTENCE_ENCODER(joke)[20],UNIVERSAL_SENTENCE_ENCODER(joke)[21],UNIVERSAL_SENTENCE_ENCODER(joke)[22],UNIVERSAL_SENTENCE_ENCODER(joke)[23],UNIVERSAL_SENTENCE_ENCODER(joke)[24],UNIVERSAL_SENTENCE_ENCODER(joke)[25],UNIVERSAL_SENTENCE_ENCODER(joke)[26],UNIVERSAL_SENTENCE_ENCODER(joke)[27],UNIVERSAL_SENTENCE_ENCODER(joke)[28],UNIVERSAL_SENTENCE_ENCODER(joke)[29],UNIVERSAL_SENTENCE_ENCODER(joke)[30],UNIVERSAL_SENTENCE_ENCODER(joke)[31],UNIVERSAL_SENTENCE_ENCODER(joke)[32],UNIVERSAL_SENTENCE_ENCODER(joke)[33],UNIVERSAL_SENTENCE_ENCODER(joke)[34],UNIVERSAL_SENTENCE_ENCODER(joke)[35],UNIVERSAL_SENTENCE_ENCODER(joke)[36],UNIVERSAL_SENTENCE_ENCODER(joke)[37],UNIVERSAL_SENTENCE_ENCODER(joke)[38],UNIVERSAL_SENTENCE_ENCODER(joke)[39],UNIVERSAL_SENTENCE_ENCODER(joke)[40],UNIVERSAL_SENTENCE_ENCODER(joke)[41],UNIVERSAL_SENTENCE_ENCODER(joke)[42],UNIVERSAL_SENTENCE_ENCODER(joke)[43],UNIVERSAL_SENTENCE_ENCODER(joke)[44],UNIVERSAL_SENTENCE_ENCODER(joke)[45],UNIVERSAL_SENTENCE_ENCODER(joke)[46],UNIVERSAL_SENTENCE_ENCODER(joke)[47],UNIVERSAL_SENTENCE_ENCODER(joke)[48],UNIVERSAL_SENTENCE_ENCODER(joke)[49],UNIVERSAL_SENTENCE_ENCODER(joke)[50],UNIVERSAL_SENTENCE_ENCODER(joke)[51],UNIVERSAL_SENTENCE_ENCODER(joke)[52],UNIVERSAL_SENTENCE_ENCODER(joke)[53],UNIVERSAL_SENTENCE_ENCODER(joke)[54],UNIVERSAL_SENTENCE_ENCODER(joke)[55],UNIVERSAL_SENTENCE_ENCODER(joke)[56],UNIVERSAL_SENTENCE_ENCODER(joke)[57],UNIVERSAL_SENTENCE_ENCODER(joke)[58],UNIVERSAL_SENTENCE_ENCODER(joke)[59],UNIVERSAL_SENTENCE_ENCODER(joke)[60],UNIVERSAL_SENTENCE_ENCODER(joke)[61],UNIVERSAL_SENTENCE_ENCODER(joke)[62],UNIVERSAL_SENTENCE_ENCODER(joke)[63],UNIVERSAL_SENTENCE_ENCODER(joke)[64],UNIVERSAL_SENTENCE_ENCODER(joke)[65],UNIVERSAL_SENTENCE_ENCODER(joke)[66],UNIVERSAL_SENTENCE_ENCODER(joke)[67],UNIVERSAL_SENTENCE_ENCODER(joke)[68],UNIVERSAL_SENTENCE_ENCODER(joke)[69],UNIVERSAL_SENTENCE_ENCODER(joke)[70],UNIVERSAL_SENTENCE_ENCODER(joke)[71],UNIVERSAL_SENTENCE_ENCODER(joke)[72],UNIVERSAL_SENTENCE_ENCODER(joke)[73],UNIVERSAL_SENTENCE_ENCODER(joke)[74],UNIVERSAL_SENTENCE_ENCODER(joke)[75],UNIVERSAL_SENTENCE_ENCODER(joke)[76],UNIVERSAL_SENTENCE_ENCODER(joke)[77],UNIVERSAL_SENTENCE_ENCODER(joke)[78],UNIVERSAL_SENTENCE_ENCODER(joke)[79],UNIVERSAL_SENTENCE_ENCODER(joke)[80],UNIVERSAL_SENTENCE_ENCODER(joke)[81],UNIVERSAL_SENTENCE_ENCODER(joke)[82],UNIVERSAL_SENTENCE_ENCODER(joke)[83],UNIVERSAL_SENTENCE_ENCODER(joke)[84],UNIVERSAL_SENTENCE_ENCODER(joke)[85],UNIVERSAL_SENTENCE_ENCODER(joke)[86],UNIVERSAL_SENTENCE_ENCODER(joke)[87],UNIVERSAL_SENTENCE_ENCODER(joke)[88],UNIVERSAL_SENTENCE_ENCODER(joke)[89],UNIVERSAL_SENTENCE_ENCODER(joke)[90],UNIVERSAL_SENTENCE_ENCODER(joke)[91],UNIVERSAL_SENTENCE_ENCODER(joke)[92],UNIVERSAL_SENTENCE_ENCODER(joke)[93],UNIVERSAL_SENTENCE_ENCODER(joke)[94],UNIVERSAL_SENTENCE_ENCODER(joke)[95],UNIVERSAL_SENTENCE_ENCODER(joke)[96],UNIVERSAL_SENTENCE_ENCODER(joke)[97],UNIVERSAL_SENTENCE_ENCODER(joke)[98],UNIVERSAL_SENTENCE_ENCODER(joke)[99],UNIVERSAL_SENTENCE_ENCODER(joke)[100],UNIVERSAL_SENTENCE_ENCODER(joke)[101],UNIVERSAL_SENTENCE_ENCODER(joke)[102],UNIVERSAL_SENTENCE_ENCODER(joke)[103],UNIVERSAL_SENTENCE_ENCODER(joke)[104],U

In [25]:
def check_missing_data(df):
  """ 
      params: dataframe
      return: data of missing values, if have some.
  """
  flag=df.isna().sum().any()
  if flag==True:
      total = df.isnull().sum()
      percent = (df.isnull().sum())*100/(df.isnull().count())
      output = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
      data_type = []
      for col in df.columns:
          dtype = str(df[col].dtype)
          data_type.append(dtype)
      output['Types'] = data_type
      return(output[output['Percent'] > 0].sort_values(by='Percent', ascending=False))
  else:
      return(False)

In [26]:
missing_X_train = check_missing_data(X_train)
missing_X_train

,Total,Percent,Types
breaking,808,20.2,float64
said like,808,20.2,float64
early,808,20.2,float64
entered,808,20.2,float64
unfortunately,808,20.2,float64
...,...,...,...
baby,808,20.2,float64
confused,808,20.2,float64
real,808,20.2,float64
fart,808,20.2,float64


In [27]:
missing_X_test = check_missing_data(X_test)
missing_X_test

,Total,Percent,Types
breaking,784,78.4,float64
said like,784,78.4,float64
early,784,78.4,float64
entered,784,78.4,float64
unfortunately,784,78.4,float64
...,...,...,...
baby,784,78.4,float64
confused,784,78.4,float64
real,784,78.4,float64
fart,784,78.4,float64


In [10]:
numerical_columns = X_train.select_dtypes(include=["int64", "float64"]).columns

In [11]:
X_train[pd.Index(numerical_columns)].shape

(4000, 1049)

In [14]:
X_train_num = X_train[pd.Index(set(numerical_columns))]
X_test_num = X_test[pd.Index(set(numerical_columns))]

In [19]:
X_train.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)

In [16]:
clf = Pipeline([
  ('feature_selection', SelectFromModel(ExtraTreesRegressor(n_estimators=50))),
  ('classification', RandomForestRegressor())
])
clf.fit(X_train_num, y_train)

Pipeline(memory=None,
         steps=[('feature_selection',
                 SelectFromModel(estimator=ExtraTreesRegressor(bootstrap=False,
                                                               ccp_alpha=0.0,
                                                               criterion='mse',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               max_samples=None,
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                                                               min_samples_leaf=1,
                                                               min_samples_split=2,
                             

In [17]:
clf.score(X_test,y_test)

ValueError: Input contains NaN